In [1]:
import numpy as np 
import pandas as pd
import itertools
from sklearn.metrics import confusion_matrix

In [2]:
def read_data():
    data_name = input('Enter the name of train data file [(ex) titanic.csv]: ')
    coding_fm = int(input("Select the data coding format (1 = 'a b c' or 2 = 'a,b,c'): "))
    separator_fm = {coding_fm == 1 : ' '}.get(True, ",")
    res_pos = int(input('Enter the column position of the response variable : [from 1 to p]:')) - 1
    algorithm = int(input('Enter the name of algorithm (1 = CART or 2 = C4.5): '))
    data_type = input('Enter the date type (a = categorical or b = numerical): ')
    if algorithm == 1:
        if data_type == 'a':
            dt = CateBinaryCart()
        else:
            dt = NumBinaryCart()
    else:
        if data_type == 'a':
            dt = CateBinaryC45()
        else:
            dt = NumBinaryCart()
    header = input('Does the data have column header? (y/n):')
    
    if header == 'y':
        data = pd.read_csv(data_name, sep=separator_fm)
        res_col = data.columns[res_pos]
        response = data[res_col]
        feature = data.drop(res_col, axis = 1)
    
    else:
        data = pd.read_csv(data_name, sep=separator_fm, header=None)
        response = data[res_pos]
        feature = data.drop(res_pos, axis = 1)
        
    out_name = input('Enter the output file name to export [(ex) result.txt]:')
    return feature, response, out_name, dt

In [3]:
class CateBinaryCart:
    def __init__(self):
        None
        
    def get_entropy(self, yes_error, no_error):
        entropy = 1 - (yes_error ** 2 + no_error ** 2)
        return entropy

    def fit(self, x, y):
        ent_lst = []
        
        for col_name in x.columns:
            x_cate = x[col_name].unique()
            weight_entropy_lst = []
            self.left_yes_cnt_lst = []
            self.left_no_cnt_lst = []
            self.right_yes_cnt_lst = []
            self.right_no_cnt_lst = []

            for i in range(len(x_cate)-1):
                ent = 1
                for c in [c for c in itertools.combinations(x_cate, i+1)]:
                    left_cond = list(c)
                    right_cond = [i for i in x_cate if not i in c]
                    
                    x_left = x[x[col_name].isin(left_cond)]
                    y_left = y[x[col_name].isin(left_cond)]
                    x_right = x[~(x[col_name].isin(left_cond))]
                    y_right = y[~(x[col_name].isin(left_cond))]
                    
                    left_yes_cnt = sum(y_left == 'Yes')
                    self.left_yes_cnt_lst.append(left_yes_cnt)
                    left_no_cnt = sum(y_left == 'No')
                    self.left_no_cnt_lst.append(left_no_cnt)
                    
                    right_yes_cnt = sum(y_right == 'Yes')
                    self.right_yes_cnt_lst.append(right_yes_cnt)
                    right_no_cnt = sum(y_right == 'No')
                    self.right_no_cnt_lst.append(right_no_cnt)
                    
                    yes_left = left_yes_cnt / len(y_left)
                    no_left = left_no_cnt / len(y_left)

                    yes_right = right_yes_cnt / len(y_right)
                    no_right = right_no_cnt / len(y_right)

                    left_entropy = self.get_entropy(yes_left, no_left)
                    right_entropy = self.get_entropy(yes_right, no_right)

                    left_w = len(x_left) / len(x)
                    right_w = len(x_right) / len(x)

                    weight_entropy = left_w * left_entropy + right_w * right_entropy
                    weight_entropy_lst.append(weight_entropy)

                    if ent > weight_entropy:
                        self.cond = left_cond
                        self.right_cond = right_cond
                        ent = weight_entropy
            ent_lst.append(ent)
            
        split_col = np.argmin(ent_lst)
        self.split_col_name = x.columns[split_col]
        self.split_col_unique = x[self.split_col_name].unique()
        
        self.yes_cnt = sum(y == 'Yes')
        self.no_cnt = sum(y == 'No')
        
        return self.split_col_name

    def predict(self, x, y):
        self.clf_lst = []
        self.pred = np.empty((x.shape[0],), object)
        
        if self.yes_cnt <= self.no_cnt:
            self.all_clf = 'No'
        else:
            self.all_clf = 'Yes'

        for val in x[self.split_col_name].unique():
            x_split_col = x[self.split_col_name]
            split_yes = (x_split_col == val) & (y == 'Yes')
            split_no = (x_split_col == val) & (y == 'No')

            if sum(split_yes) <= sum(split_no):
                self.pred[x_split_col == val] = 'No'
                clf = 'No'
                
            else:
                self.pred[x_split_col == val] = 'Yes'
                clf = 'Yes'
            self.clf_lst.append(clf)

        return self.pred

    def accuracy(self, y):
        acc = np.mean(self.pred == y)
        return acc
    
    def evaluate_print(self, x, y, out_name):
        text = f'''Tree Structure (CART)
    Node 1: {self.all_clf} ({self.yes_cnt}, {self.no_cnt})
        Node 2: Class = {self.cond}, {self.clf_lst[0]}, ({self.left_yes_cnt_lst[0]}, {self.left_no_cnt_lst[0]})
        Node 3: Class = {self.right_cond}, {self.clf_lst[1]}, ({self.left_yes_cnt_lst[1]}, {self.left_no_cnt_lst[1]})

Confusion Matrix (Training)
---------------------------
{confusion_matrix(y, self.predict(x, y))}

Model Summary (Training)
------------------------
Overall accuracy = {self.accuracy(y):.3f}
'''
        print(text)
        file = open(out_name, "w") 
        file.write(text)
        file.close()

In [4]:
class CateBinaryC45:
    def __init__(self):
        None
        
    def get_entropy(self, yes_error, no_error):
        self.entropy = - (yes_error * np.log2(yes_error) + no_error * np.log2(no_error))
        return self.entropy

    def fit(self, x, y):
        entropy_lst = []
        column_cnt_lst = []

        for col_name in x.columns:
            column_entropy = 0
            self.yes_cnt_lst = []
            self.no_cnt_lst = []

            x_cate = x[col_name].unique()
            y_cate = y.unique()

            for idx, val in enumerate(x_cate):
                total_cnt_node = sum(x[col_name] == val)
                yes_cnt_node = sum((x[col_name] == val) & (y == 'Yes'))
                self.yes_cnt_lst.append(yes_cnt_node)
                no_cnt_node = total_cnt_node - yes_cnt_node
                self.no_cnt_lst.append(no_cnt_node)

                yes_error = yes_cnt_node / total_cnt_node
                no_error = 1 - yes_error
                weight = sum((x[col_name] == val)) / len(x)

                entropy = self.get_entropy(yes_error, no_error)
                column_entropy += weight * entropy
            entropy_lst.append(column_entropy)
            
        self.yes_cnt = sum(y == 'Yes')
        self.no_cnt = sum(y == 'No')
        
        split_col = np.argmin(entropy_lst)
        self.split_col_name = x.columns[split_col]
        self.split_col_unique = x[self.split_col_name].unique()

    def predict(self, x, y):
        self.clf_lst = []
        self.pred = np.empty((x.shape[0],), object)
        
        if self.yes_cnt <= self.no_cnt:
            self.all_clf = 'No'
        else:
            self.all_clf = 'Yes'

        for val in x[self.split_col_name].unique():
            x_split_col = x[self.split_col_name]
            split_yes = (x_split_col == val) & (y == 'Yes')
            split_no = (x_split_col == val) & (y == 'No')

            if sum(split_yes) <= sum(split_no):
                self.pred[x_split_col == val] = 'No'
                clf = 'No'
                
            else:
                self.pred[x_split_col == val] = 'Yes'
                clf = 'Yes'
            self.clf_lst.append(clf)

        return self.pred

    def accuracy(self, y):
        acc = np.mean(self.pred == y)
        return acc
    
    def evaluate_print(self, x, y, out_name):
        text = f'''Tree Structure (C4.5)
    Node 1: {self.all_clf} ({self.yes_cnt}, {self.no_cnt})
        Node 2: Class = {self.split_col_unique[0]}, {self.clf_lst[0]}, ({self.yes_cnt_lst[0]}, {self.no_cnt_lst[0]})
        Node 3: Class = {self.split_col_unique[1]}, {self.clf_lst[1]}, ({self.yes_cnt_lst[1]}, {self.no_cnt_lst[1]})

Confusion Matrix (Training)
---------------------------
{confusion_matrix(y, self.predict(x, y))}

Model Summary (Training)
------------------------
Overall accuracy = {self.accuracy(y):.3f}
'''
        print(text)
        file = open(out_name, "w") 
        file.write(text)
        file.close()

In [5]:
class NumBinaryCart:
    def __init__(self):
        None

    def get_gini(self, y):
        p = np.unique(y, return_counts=True)[1] / len(y)
        t = 1 - np.sum(p ** 2)
        return t

    def get_split(self, sort_x, sort_y):
        fit_idx = 1000
        fit_gini = 2
        fit_y = None
        pre_x = sort_x[0]
        
        for idx, x_val in enumerate(sort_x):
            if x_val != pre_x:
                y_left = sort_y[:idx]
                y_right = sort_y[idx:]

                t1 = self.get_gini(y_left)
                t2 = self.get_gini(y_right)
                
                left_weight = len(y_left) / len(sort_y)
                right_weight = len(y_right) / len(sort_y)
                
                gini = left_weight * t1 + right_weight * t2

                if gini < fit_gini:
                    fit_idx = idx 
                    fit_gini = gini
                    left_y, left_cnt = self.get_node(y_left)
                    right_y, right_cnt = self.get_node(y_right)
                    fit_y = ((left_y, left_cnt), (right_y, right_cnt))
                    
                pre_x = x_val
                
        return fit_idx, fit_gini, fit_y


    def get_node(self, y):
        labels, cnts = np.unique(y, return_counts=True)
        return labels[cnts.argmax()], cnts
    
    def fit(self, x, y):
        col_num = x.shape[1]
        split_lst = []

        for i in range(col_num):
            x_col = x.values[:, i]
            sort_x = x_col[x_col.argsort()]
            sort_y = y[x_col.argsort()]
            
            fit_idx, fit_score, fit_y = self.get_split(sort_x, sort_y)
            split_pt = (sort_x[fit_idx-1] + sort_x[fit_idx])/2
            
            split_lst.append((fit_idx, fit_score, fit_y, split_pt))

        self.best_x_idx = np.argmin([x[1] for x in split_lst], axis=0)
        self.best_idx, self.best_gini, fit_y, self.best_split = split_lst[self.best_x_idx]
        
        (self.left_y, self.left_cnt), (self.right_y, self.right_cnt) = fit_y
        self.y, self.cnt = self.get_node(y)

    def predict(self, x):
        pred = np.zeros(x.shape[0],)
        
        left = x[self.best_x_idx] <= self.best_split
        right = x[self.best_x_idx] > self.best_split
        
        pred[left] = self.left_y
        pred[right] = self.right_y
        return pred
    
    def accuracy(self, y, pred_y):
        acc = np.mean(pred_y == y.values)
        return acc
    
    def evaluate_print(self, x, y, out_name):
        text = f'''Tree Structure
    Node 1: {dt.y} ({dt.cnt[0]}, {dt.cnt[1]})
    Node 2: x{dt.best_x_idx+1} <= {dt.best_split}, {dt.left_y} ({dt.left_cnt[0]}, {dt.left_cnt[1]})
    Node 3: x{dt.best_x_idx+1} > {dt.best_split}, {dt.right_y} ({dt.right_cnt[0]}, {dt.right_cnt[1]})
    
Confusion Matrix (Test)
-----------------------
{confusion_matrix(tst_y, dt.predict(tst_x))}

Model Summary (Test)
--------------------
Overall accuracy = {dt.accuracy(tst_y, dt.predict(tst_x)):.3f}
'''
        print(text)
        file = open(out_name, "w") 
        file.write(text)
        file.close()

In [6]:
class NumBinaryC45:
    def __init__(self):
        None

    def get_gini(self, y):
        p = np.unique(y, return_counts=True)[1] / len(y)
        t = - np.sum(p * np.log2(p))
        return t

    def get_split(self, sort_x, sort_y):
        fit_idx = 1000
        fit_gini = 2
        fit_y = None
        pre_x = sort_x[0]
        
        for idx, x_val in enumerate(sort_x):
            if x_val != pre_x:
                y_left = sort_y[:idx]
                y_right = sort_y[idx:]

                t1 = self.get_gini(y_left)
                t2 = self.get_gini(y_right)
                
                left_weight = len(y_left) / len(sort_y)
                right_weight = len(y_right) / len(sort_y)
                
                gini = left_weight * t1 + right_weight * t2

                if gini < fit_gini:
                    fit_idx = idx 
                    fit_gini = gini
                    left_y, left_cnt = self.get_node(y_left)
                    right_y, right_cnt = self.get_node(y_right)
                    fit_y = ((left_y, left_cnt), (right_y, right_cnt))
                    
                pre_x = x_val
                
        return fit_idx, fit_gini, fit_y


    def get_node(self, y):
        labels, cnts = np.unique(y, return_counts=True)
        return labels[cnts.argmax()], cnts
    
    def fit(self, x, y):
        col_num = x.shape[1]
        split_lst = []

        for i in range(col_num):
            x_col = x.values[:, i]
            sort_x = x_col[x_col.argsort()]
            sort_y = y[x_col.argsort()]
            
            fit_idx, fit_score, fit_y = self.get_split(sort_x, sort_y)
            split_pt = (sort_x[fit_idx-1] + sort_x[fit_idx])/2
            
            split_lst.append((fit_idx, fit_score, fit_y, split_pt))

        self.best_x_idx = np.argmin([x[1] for x in split_lst], axis=0)
        self.best_idx, self.best_gini, fit_y, self.best_split = split_lst[self.best_x_idx]
        
        (self.left_y, self.left_cnt), (self.right_y, self.right_cnt) = fit_y
        self.y, self.cnt = self.get_node(y)

    def predict(self, x):
        pred = np.zeros(x.shape[0],)
        
        left = x[self.best_x_idx] <= self.best_split
        right = x[self.best_x_idx] > self.best_split
        
        pred[left] = self.left_y
        pred[right] = self.right_y
        return pred
    
    def accuracy(self, y, pred_y):
        acc = np.mean(pred_y == y.values)
        return acc
    
    def evaluate_print(self, x, y, out_name):
        text = f'''Tree Structure
    Node 1: {dt.y} ({dt.cnt[0]}, {dt.cnt[1]})
        Node 2: x{dt.best_x_idx+1} <= {dt.best_split}, {dt.left_y} ({dt.left_cnt[0]}, {dt.left_cnt[1]})
        Node 3: x{dt.best_x_idx+1} > {dt.best_split}, {dt.right_y} ({dt.right_cnt[0]}, {dt.right_cnt[1]})

Confusion Matrix (Test)
-----------------------
{confusion_matrix(tst_y, dt.predict(tst_x))}

Model Summary (Test)
--------------------
Overall accuracy = {dt.accuracy(tst_y, dt.predict(tst_x)):.3f}
'''
        print(text)
        file = open(out_name, "w") 
        file.write(text)
        file.close()

In [7]:
x, y, out_name, dt = read_data()

Enter the name of train data file [(ex) titanic.csv]: titanic.csv
Select the data coding format (1 = 'a b c' or 2 = 'a,b,c'): 2
Enter the column position of the response variable : [from 1 to p]:4
Enter the name of algorithm (1 = CART or 2 = C4.5): 1
Enter the date type (a = categorical or b = numerical): a
Does the data have column header? (y/n):y
Enter the output file name to export [(ex) result.txt]:result(cart).txt


In [8]:
dt.fit(x,y)

'Sex'

In [9]:
dt.predict(x,y)

array(['No', 'No', 'No', ..., 'Yes', 'Yes', 'Yes'], dtype=object)

In [10]:
dt.accuracy(y)

0.7760109041344844

In [11]:
dt.evaluate_print(x, y, out_name)

Tree Structure (CART)
    Node 1: No (711, 1490)
        Node 2: Class = ['Male'], No, (367, 1364)
        Node 3: Class = ['Female'], Yes, (344, 126)

Confusion Matrix (Training)
---------------------------
[[1364  126]
 [ 367  344]]

Model Summary (Training)
------------------------
Overall accuracy = 0.776

